In [1]:
# Extract the data from the Mauritius harvested files to compatible format for the visualizer

In [2]:
import os,sys
import pandas as pd
import numpy as np 
import h5py as h5
from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_2894138/2179053797.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
# inter_files = [data+'/'+f for f in os.listdir('/data') if 'intercomparison' in f ]

### document
https://wmoomm.sharepoint.com/sites/wmocpdb/eve_activityarea/Forms/AllItems.aspx?id=%2Fsites%2Fwmocpdb%2Feve%5Factivityarea%2FInstruments%20and%20Methods%20of%20Observation%20Programme%20%28IMOP%29%5F67452102%2D7575%2De911%2Da98e%2D000d3a44bd9c%2FIntercomparisons%2FPast%20Intercomparisons%2FRSO%2DIC%2D2005%5FFinal%5FReport%2Epdf&parent=%2Fsites%2Fwmocpdb%2Feve%5Factivityarea%2FInstruments%20and%20Methods%20of%20Observation%20Programme%20%28IMOP%29%5F67452102%2D7575%2De911%2Da98e%2D000d3a44bd9c%2FIntercomparisons%2FPast%20Intercomparisons&p=true&ga=1


In [4]:
digitized = '/scratch/das/federico/databases_service2/MAURITIUS/'


In [5]:
### create a single file for each sonde type

# extracting the list of different sensor types
sonde_types = np.unique( [ f.split('_')[1].split('.csv')[0] for f in os.listdir(digitized+'/temp/') if '.csv' in f  ])
sonde_types

array(['Graw', 'Graw-GPS', 'MKII', 'Meisei', 'Modem', 'SRS', 'Sip',
       'Vaisala', 'Vaisala-GPS'], dtype='<U11')

In [6]:
# creating single files out of the single ascent files, per sensor 

# output directory for the processed files 
outdir = 'data_processed'
if not os.path.isdir(outdir):
    os.mkdir(outdir)
    

In [7]:
os.path.isdir(outdir)

True

In [8]:
os.getcwd()

'/mnt/users/staff/federico/GitHub/CEUAS_master_JULY2922/CEUAS/CEUAS/papers/mauritius'

In [13]:
# creating single files out of the single ascent files, per sensor 

# output directory for the processed files 
outdir = 'data_processed'
if not os.path.isdir(outdir):
    os.mkdir(outdir)

#sonde_types = ['Vaisala']

# looping over each distinct sensor, all the ascents
for s in sonde_types:

    ### temperature
    print('+++ Extracting temperature data +++ ')
    all_df_temp = []
    files_temp = [ digitized + '/temp/' + f for f in os.listdir(digitized+'/temp/') if f.split('_')[1].split('.csv')[0]== s ]  
    files_temp.sort()
    for f in tqdm(files_temp):
        df_temp = pd.read_csv(f , sep = ',' )
        df_temp = df_temp[ [c for c in df_temp.columns if c in ['press','datetime','temp','hum']] ] 
        all_df_temp.append(df_temp)

    df_sum_temp = pd.concat(all_df_temp)
    df_sum_temp = df_sum_temp.round(decimals=2)
    df_sum_temp['datetime'] = pd.to_datetime(df_sum_temp.datetime,  utc=True)
    df_sum_temp = df_sum_temp.dropna(subset=['press'])
    df_sum_temp['press'] = df_sum_temp.press.astype(int)

    df_sum_temp.to_csv( outdir + '/' + s + '_all_ascents_temp.csv' , sep ='\t' )

    
    ### humidity    
    print('+++ Extracting humidity data +++ ')
    all_df_hum = []
    files_hum = [ digitized + '/hum/' + f for f in os.listdir(digitized+'/hum/') if f.split('_')[1].split('.csv')[0]== s ]
    files_hum.sort()
    for f in tqdm(files_hum):
        df_hum = pd.read_csv(f , sep = ',' )
        df_hum = df_hum[ [c for c in df_hum.columns if c in ['press','datetime','temp','hum']] ] 
        all_df_hum.append(df_hum)

    df_sum_hum = pd.concat(all_df_hum)
    df_sum_hum = df_sum_hum.round(decimals=2)
    df_sum_hum['datetime'] = pd.to_datetime(df_sum_hum.datetime, utc=True )
    df_sum_hum = df_sum_hum.dropna(subset=['press'])
    df_sum_hum['press'] = df_sum_hum.press.astype(int)
    df_sum_hum.to_csv( outdir + '/' + s + '_all_ascents_hum.csv' , sep ='\t' )
    
    
    # merging temp and hum data in one single file per sonde 
    all_times = list( np.unique(df_sum_temp.datetime) ) + list( np.unique(df_sum_hum.datetime) ) 

    all_df = []
    for dt in all_times:
        hum = df_sum_hum.loc[df_sum_hum['datetime'] == dt ][['press','hum']]
        temp = df_sum_temp.loc[df_sum_temp['datetime'] == dt ][['press','temp']]

        mer= pd.merge(hum,temp, on='press', how='outer')

        #mer = pd.concat( [ hum.join(temp, on='press'), temp.join(hum, on='press') ]  ).drop_duplicates(subset=['press'] )
        #mer = mer[['press','hum','temp']]
        mer['datetime'] = dt 
        all_df.append(mer)


    df = pd.concat(all_df)
    df = df.sort_values(by=['datetime','press']) 
    df.to_csv( outdir + '/' + s + '_all_ascents_fulldata.csv' , sep ='\t' )



+++ Extracting temperature data +++ 


100%|██████████| 29/29 [00:01<00:00, 21.67it/s]


+++ Extracting humidity data +++ 


100%|██████████| 31/31 [00:01<00:00, 18.07it/s]


+++ Extracting temperature data +++ 


...

100%|██████████| 61/61 [00:04<00:00, 13.93it/s]


+++ Extracting temperature data +++ 


100%|██████████| 61/61 [00:02<00:00, 26.79it/s]


+++ Extracting humidity data +++ 


100%|██████████| 61/61 [00:03<00:00, 15.25it/s]


In [12]:
len(df_sum_temp)
df_sum_temp.head(5)

,press,temp,datetime
0,431,300.48,2005-07-02 10:01:53.551000+00:00
1,435,298.65,2005-07-02 10:01:53.551000+00:00
2,439,300.80,2005-07-02 10:01:53.551000+00:00
3,443,299.38,2005-07-02 10:01:53.551000+00:00
4,448,298.38,2005-07-02 10:01:53.551000+00:00


In [ ]:
df_sum_temp.datetime

In [ ]:
all_times = list( np.unique(df_sum_temp.datetime) ) + list( np.unique(df_sum_hum.datetime) ) 

In [ ]:
all_times[:5]

In [ ]:
#len(df_sum_hum)
#df_sum_hum.head(5)

In [ ]:
all_df = []
for dt in all_times:
    hum = df_sum_hum.loc[df_sum_hum['datetime'] == dt ]
    temp = df_sum_temp.loc[df_sum_temp['datetime'] == dt ]

    mer= pd.merge(a,b, on='press', how='outer')
    mer = mer[['press','hum','temp']]
    mer['datetime'] = dt 
    all_df.append(mer)




In [ ]:
df = pd.concat(all_df)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
#hum = df_sum_hum.loc[df_sum_hum['datetime'] == all_times[3] ][['hum','press']] [:100]
#temp = df_sum_temp.loc[df_sum_temp['datetime'] == all_times[3] ][['temp','press']] [:100]

In [ ]:

#mer= pd.merge(hum, temp , on='press', how='outer')
#mer2= pd.merge(temp, hum , on='press', how='outer')


In [ ]:
#df_join = df_sum_hum.join(df_sum_temp,  lsuffix='l' )

In [ ]:
#df_sum_hum = df_sum_hum.set_index('datetime')
#df_sum_temp = df_sum_temp.set_index('datetime')


#a = df_sum_hum[:100]
#b = df_sum_temp[:100]

#a['press'] = a.press.astype(int)
#b['press'] = b.press.astype(int)

#a = a.set_index(['press', 'datetime'])
#b = b.set_index(['press', 'datetime'])

#df_join = a.join(b)